In [1]:
connecting_letters = [
    'ب', 'ت', 'ث', 'ج', 'ح', 'خ',
    'س', 'ش', 'ص', 'ض', 'ط', 'ظ',
    'ع', 'غ', 'ف', 'ق', 'ك', 'ل',
    'م', 'ن', 'ه', 'ي'
]
non_connecting_letters = ['ا', 'أ', 'إ', 'آ', 'د', 'ذ', 'ر', 'ز', 'و', 'ؤ', 'ء', 'ئ', 'ى', 'ة']

arabic_letters = connecting_letters + non_connecting_letters

replacements = {
    'isolated': {
        'ب': 'ٮ', 'ت': 'ٮ', 'ث': 'ٮ', 'ج': 'ح', 'خ': 'ح',
        'ذ': 'د', 'ز': 'ر', 'ش': 'س', 'ض': 'ص', 'ظ': 'ط',
        'غ': 'ع', 'ف': 'ڡ', 'ق': 'ٯ', 'ن': 'ں', 'ي': 'ى', 'ة': 'ه'
    },
    
    
    'initial': {
        'ب': 'ٮ', 'ت': 'ٮ', 'ث': 'ٮ', 'ج': 'ح', 'خ': 'ح',
        'ذ': 'د', 'ز': 'ر', 'ش': 'س', 'ض': 'ص', 'ظ': 'ط', 
        'غ': 'ع', 'ف': 'ڡ', 'ق': 'ڡ', 'ن': 'ٮ', 'ي': 'ٮ', 'ة': 'ه'
    },
    
    # ي → ٮ in medial
    'medial': {
        'ب': 'ٮ', 'ت': 'ٮ', 'ث': 'ٮ', 'ج': 'ح', 'خ': 'ح',
        'ذ': 'د', 'ز': 'ر', 'ش': 'س', 'ض': 'ص', 'ظ': 'ط', 
        'غ': 'ع', 'ف': 'ڡ', 'ق': 'ڡ', 'ن': 'ٮ', 'ي': 'ٮ', 'ة': 'ه'
    },
    
    # ي → ى in final
    'final': {
        'ب': 'ٮ', 'ت': 'ٮ', 'ث': 'ٮ', 'ج': 'ح', 'خ': 'ح',
        'ذ': 'د', 'ز': 'ر', 'ش': 'س', 'ض': 'ص', 'ظ': 'ط',
        'غ': 'ع', 'ف': 'ڡ', 'ق': 'ٯ', 'ن': 'ں', 'ي': 'ى', 'ة': 'ه'  
    }
}


def remove_dots(sentence): 
    padded_sentence = f' {sentence} '

    new_sentence = ''
    for i in range(1, len(padded_sentence) - 1):
        if padded_sentence[i - 1] in connecting_letters:
            if padded_sentence[i + 1] in arabic_letters:
                new_sentence += replacements['medial'].get(padded_sentence[i], padded_sentence[i])  # medial
            else:
                new_sentence += replacements['final'].get(padded_sentence[i], padded_sentence[i])  # final
        else: 
            if padded_sentence[i + 1] in arabic_letters:
                new_sentence += replacements['initial'].get(padded_sentence[i], padded_sentence[i])  # initial
            else:
                new_sentence += replacements['isolated'].get(padded_sentence[i], padded_sentence[i])  # isolated 
    return new_sentence

sentence =  "زينة"
remove_dots(sentence)


'رٮٮه'

In [2]:
from datasets import load_dataset
 
dataset = load_dataset("wikimedia/wikipedia", "20231101.ar", split="train", streaming=True)  

# Take just a few samples
from itertools import islice
small_dataset = list(islice(dataset, 20000))  


In [3]:
text_data = [d['text'] for d in small_dataset]


In [4]:
# Preprocess Text
import re

def clean_arabic_text(text):
    # Remove Arabic diacritics (harakat)
    harakat = r'[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]'
    text = re.sub(harakat, '', text)

    # Keep only Arabic letters, numbers, special characters, and parentheses
    pattern = r'[^\u0600-\u06FF0-9\(\)\[\]\{\}.,:;!?؟%\-+=*/\\@#&<>\"\'\s]'
    text = re.sub(pattern, '', text)
    
    return re.sub('\n+', ' ', text)


cleaned_text_data = [clean_arabic_text(text) for text in text_data]

In [5]:
from random import randint
# subtext
max_sequence_length = 300

sampled_text_data = []

for text in cleaned_text_data:
    tokenized_text = text.split(' ')
    
    if len(tokenized_text) < max_sequence_length:
        continue
        
    random_start_index = randint(0, len(tokenized_text) - max_sequence_length)
    
    sampled_text = ' '.join(tokenized_text[random_start_index: random_start_index + max_sequence_length])
     
    sampled_text_data.append(sampled_text)
    

In [6]:
# generate the dotless dataset

dotless_text_data = [remove_dots(text) for text in sampled_text_data]

In [7]:
sampled_text_data[1]

'معادلة شرودنغر المصادر والمراجع مصادر هوامش  . علم الفضاء والعدد والكمية والترتيب، الذي تتضمن طرائقه التفكير المنطقي واستخدام الترميز الرمزي، والذي يتضمن الهندسة والحساب والجبر والتحليل. . الرياضيات... هي ببساطة دراسة الهياكل المجردة، أو الأنماط الرسمية للترابط. . التفاضل والتكامل هو دراسة التغيير، كيف تتغير الأشياء، ومدى سرعة تغيرها. . الاقتباس هو إجابة أينشتاين على السؤال: كيف يمكن أن تكون الرياضيات، كونها نتاج فكر بشري مستقل عن التجربة، مناسبة بشكل مثير للإعجاب لموضوعات الواقع؟ استلهم هذا السؤال من مقالة يوجين وينر الفعالية غير المعقولة للرياضيات في العلوم الطبيعية. . من الضروري أولا أن نسأل ما هو المقصود بـالرياضيات بشكل عام. ناقش العلماء اللامعون هذه المسألة حتى أصبح وجههم أزرق، ومع ذلك لم يتم التوصل إلى إجماع حول ما إذا كانت الرياضيات هي علم طبيعي، أو فرع من العلوم الإنسانية، أو شكلا من أشكال الفن. . غالبا ما يكون حل المشكلات المركزة في الرياضيات والعلوم أكثر جهدا من التفكير المركز الذي يشمل اللغة والناس. قد يكون هذا بسبب عدم تطور البشر على مدى آلاف السنين للتلاعب بالأفكار الريا

In [8]:
# save the dataset

with open('original_text.txt', 'w', encoding='utf-8') as f:
    for text in sampled_text_data:
        f.write(text + '\n')
    

with open('dotless_text.txt', 'w', encoding='utf-8') as f: 
    for text in dotless_text_data:
        f.write(text + '\n')